# Case study

In the case study we demonstrate how to generate counterfactual explanations by using our library on Statlog (German Credit Data) Data Set from UCI ML repository.  

In [1]:
from tensorflow import keras
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
np.random.seed(44)

### Data and model loading

Statlog (German Credit Data) was gathered from UCI ML repository and consists of 20 features and 61 columns (most of them are represented by one-hot encoding). The dataset is wrapped in GermanData class, for easier use. 

Additionally, we load a pretrained keras model (simple logistic regression with 2 outputs) which will be used for prediction making.

In [2]:
from data import GermanData

german_data = GermanData('data/datasets/input_german.csv', 'data/datasets/labels_german.csv')
model = keras.models.load_model('models/model_german')

In [3]:
german_data.input.sample(5)

,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
99.0,20.0,7057.0,3.0,4.0,36.0,2.0,2.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
488.0,10.0,1418.0,3.0,2.0,35.0,1.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
206.0,12.0,1935.0,4.0,4.0,43.0,3.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
939.0,24.0,6842.0,2.0,4.0,46.0,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
729.0,24.0,1275.0,2.0,4.0,36.0,2.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


### Test sample selection

We select one instance, for which we want to calculate the counterfactual. 

In [4]:
X_test = german_data.input.iloc[0]
X_test

duration                    6.0
credit                   1169.0
installment_percent         4.0
residence_duration          4.0
age                        67.0
                          ...  
employment_1..4 years       0.0
employment_4..7 years       0.0
employment_< 1 year         0.0
employment_>= 7 years       1.0
employment_unemployed       0.0
Name: 0.0, Length: 61, dtype: float64

### Data scaling

For the model to work, the dataset has to be standardized. The GermanData scale function uses StandardScaler from sklearn.

In [5]:
X_test_scaled = german_data.scale(X_test)
X_test_scaled

duration                 0.029412
credit                   0.050567
installment_percent      1.000000
residence_duration       1.000000
age                      0.857143
                           ...   
employment_1..4 years    0.000000
employment_4..7 years    0.000000
employment_< 1 year      0.000000
employment_>= 7 years    1.000000
employment_unemployed    0.000000
Length: 61, dtype: float64

Now, we evaluate the model on the test sample.

In [6]:
model.predict(np.expand_dims(X_test_scaled, axis=0))

1/1 [==============================] - 0s 127ms/step


array([[0.00939065, 0.9906094 ]], dtype=float32)

These outputs  are interpreted as the model prediction of the testing instance to class 1. For this credit data it means a bad (not paying loans) class of customers.

### Counterfactual explanations generation

Here we demonstrate how to generate counterfactual explanations using CADEX, FIMAP and ECE methods implemented in our library.

#### FIMAP


In [22]:
from cfec.explainers import Fimap

In [24]:

model_predictions = model.predict(german_data.X_train)
model_predictions = np.argmax(model_predictions, axis=1)
fimap = Fimap()
fimap.fit(german_data.X_train, model_predictions)

cf_fimap = fimap.generate(X_test)

25/25 [==============================] - 0s 2ms/step

Training s
Training loss (for one batch): 0.2006 
Training accuracy 0.9198125

Training g
Training loss (for one batch): 0.9796 
Training accuracy 0.07875
Training loss (for one batch): 0.9525 
Training accuracy 0.07875


In [25]:
model.predict(german_data.scale(cf_fimap))

1/1 [==============================] - 0s 19ms/step


array([[1., 0.]], dtype=float32)

The class predicted for the counterfactual is 0, meaning good credit score.

In [26]:
from cfec.visualization import show

show(X_test, cf_fimap)

,X,X',change
duration,6.0,-3.526000,-9.526000
credit,1169.0,1153.064941,-15.935059
installment_percent,4.0,-50.490002,-54.490002
residence_duration,4.0,-34.064999,-38.064999
age,67.0,-4.004000,-71.004000
...,...,...,...
employment_1..4 years,0.0,-79.759003,-79.759003
employment_4..7 years,0.0,-1.870000,-1.870000
employment_< 1 year,0.0,48.715000,48.715000
employment_>= 7 years,1.0,-61.115002,-62.115002


We can see two things wrong with the generated counterfactual: 
- the categorical variables, originally represented in one-hot encoding, were changed to values different than 0 or 1
- the value of age variable decreased, resulting in poor quality of the counterfactual - we can't recommend someone to decrease their age in order to obtain credit

To fix it we can use constraints, which can be defined either in code or in spreadsheets (which can be used by users not familiar with programming).

In [27]:
from cfec.constraints import OneHot, ValueMonotonicity

constraints = [OneHot("account_status", 7, 10), 
               OneHot("credit_history", 11, 15),
               OneHot("purpose", 16, 25), 
               OneHot("savings", 26, 30), 
               OneHot("sex_status", 31, 34),
               OneHot("debtors", 35, 37), 
               OneHot("property", 38, 41),
               OneHot("other_installment_plans", 42, 44), 
               OneHot("housing", 45, 47), 
               OneHot("job", 48, 51),
               OneHot("phone", 52, 53), 
               OneHot("foreign", 54, 55), 
               OneHot("employment", 56, 60),
               ValueMonotonicity(['age'], "increasing")
              ]

In [28]:
fimap = Fimap(constraints=constraints, use_mapper=True)
fimap.fit(german_data.X_train, model_predictions)

cf_fimap_constraints = fimap.generate(X_test)

Continous columns: ['duration', 'credit', 'installment_percent', 'residence_duration', 'age', 'existing_credits', 'people_maintained', 'account_status_0..200 DM', 'account_status_< 0 DM', 'account_status_>= 200 DM', 'account_status_no checking account', 'credit_history_all credits at this bank paid back duly', 'credit_history_critical account', 'credit_history_delay in paying off in the past', 'credit_history_existing credits paid back duly till now', 'credit_history_no credits taken', 'purpose_business', 'purpose_car (new)', 'purpose_car (used)', 'purpose_domestic appliances', 'purpose_education', 'purpose_furniture/equipment', 'purpose_others', 'purpose_radio/television', 'purpose_repairs', 'purpose_retraining', 'savings_100..500 DM', 'savings_500..1000 DM', 'savings_< 100 DM', 'savings_>= 1000 DM', 'savings_unknown/ no savings account', 'sex_status_female: divorced/separated/married', 'sex_status_male: divorced/separated', 'sex_status_male: married/widowed', 'sex_status_male: single

In [29]:
model.predict(german_data.scale(cf_fimap_constraints))

1/1 [==============================] - 0s 20ms/step


array([[1.000000e+00, 1.599048e-18]], dtype=float32)

As FIMAP is a method that doesn't guarantee finding a true counterfactual (with a change in prediction), we should always check whether the prediction changed. Here, we'd have to tune the hyperparameters to find a true counterfactual.

In [30]:
cf_fimap_constraints.squeeze()


duration                  128.459567
credit                   1217.464041
installment_percent       169.450522
residence_duration        181.947724
age                        67.000001
                            ...     
employment_1..4 years       0.805302
employment_4..7 years       0.172500
employment_< 1 year         0.171250
employment_>= 7 years       0.262500
employment_unemployed       0.060000
Name: 0, Length: 61, dtype: float64

In [31]:
# from cfec.visualization import show
# cf = cf_fimap_constraints.squeeze().round(3)
# x = X_test.round(3)
# df = pd.concat([x, cf.transpose()], axis=1)
# df.columns = ["X", "X'"]
# df["index"] = list(range(len(x)))
# df = df[df["X"] != df["X'"]]
# df["change"] = df["X'"] - df["X"]

# for constraint in constraints:
#     if isinstance(constraint, OneHot):
#         changed = df[df["index"].between(constraint.start_column, constraint.end_column)]
#         if len(changed) == 2:
#                 print(changed)
#                 value_original = changed["X"][changed["X"] == 1].index.tolist()[0]
#                 print(value_original)
#                 value_cf = changed["X'"][changed["X'"] == 1].index.tolist()[0]
#                 df.loc[constraint.name] = [value_original, value_cf, -1, value_cf, "OneHot"]
#                 df.drop(changed.index, inplace=True)


In [32]:
show(X_test, cf_fimap_constraints.squeeze(), constraints=constraints)

,X,X',change,constraint
duration,6.0,128.460,122.460,
credit,1169.0,1217.464,48.464,
installment_percent,4.0,169.451,165.451,
residence_duration,4.0,181.948,177.948,
existing_credits,2.0,-48.084,-50.084,
people_maintained,1.0,88.019,87.019,
account_status_0..200 DM,0.0,0.275,0.275,
account_status_< 0 DM,1.0,0.279,-0.721,
account_status_>= 200 DM,0.0,0.068,0.068,
account_status_no checking account,0.0,0.864,0.864,


In [33]:
X_test

duration                    6.0
credit                   1169.0
installment_percent         4.0
residence_duration          4.0
age                        67.0
                          ...  
employment_1..4 years       0.0
employment_4..7 years       0.0
employment_< 1 year         0.0
employment_>= 7 years       1.0
employment_unemployed       0.0
Name: 0.0, Length: 61, dtype: float64

The constraints column shows what constraints have been placed on the given attribute. If these constraints were not met, it would be marked with an asterisk.

#### CADEX

For CADEX, we can either pass scaled instance and then unscale the obtained counterfactual or pass transform and inverse_transform parameters to the constructor.

In [34]:
from cfec.explainers import Cadex 

cadex = Cadex(model, transform=german_data.scale, inverse_transform=german_data.unscale, n_changed=10)
cf = cadex.generate(X_test)

In [35]:
model.predict(german_data.scale(cf))

1/1 [==============================] - 0s 19ms/step


array([[0.503318, 0.496682]], dtype=float32)

In [36]:
from cfec.visualization import show
show(X_test, cf)

,X,X',change
credit,1169.0,13925.251,12756.251
account_status_no checking account,0.0,-0.702,-0.702
credit_history_all credits at this bank paid back duly,0.0,0.702,0.702
purpose_domestic appliances,0.0,-0.702,-0.702
purpose_radio/television,1.0,0.298,-0.702
savings_>= 1000 DM,0.0,-0.702,-0.702
savings_unknown/ no savings account,1.0,0.298,-0.702
debtors_guarantor,0.0,-0.702,-0.702
property_real estate,1.0,0.298,-0.702
housing_own,1.0,0.298,-0.702


For CADEX we can also use constraints:

In [37]:
cadex = Cadex(model, n_changed=10, transform=german_data.scale, inverse_transform=german_data.unscale, constraints=constraints)
cf = cadex.generate(X_test)

In [38]:
model.predict(german_data.scale(cf))

1/1 [==============================] - 0s 18ms/step


array([[0.7723859 , 0.22761413]], dtype=float32)

In [39]:
show(X_test, cf, constraints=constraints)

,X,X',change,constraint
credit,1169.0,10486.206,9317.206,
existing_credits,2.0,0.462,-1.538,
credit_history_all credits at this bank paid back duly,0.0,1.000,1.000,
credit_history_critical account,1.0,0.000,-1.000,
purpose_education,0.0,1.000,1.000,
purpose_radio/television,1.0,0.000,-1.000,
savings_100..500 DM,0.0,1.000,1.000,
savings_unknown/ no savings account,1.0,0.000,-1.000,
property_real estate,1.0,0.000,-1.000,
property_unknown / no property,0.0,1.000,1.000,


### ECE 
We can use ECE to select the best counterfactuals - we'll run it using 10 explainers, 5 FIMAPs and 5 CADEXs with different parameter values. 

In [41]:
fimaps = []
fimap_hyperparameters = [
    (0.1, 0.001, 0.01),
    (0.1, 0.05, 0.5),
    (0.2, 0.01, 0.1),
    (0.2, 0.08, 0.8),
    (0.5, 0.001, 0.01)
]
for tau, l1, l2 in fimap_hyperparameters:
    fimap = Fimap(tau, l1, l2)
    fimap.fit(german_data.X_train, model_predictions)
    fimaps.append(fimap)
    
cadexs = []
n_list = [5, 8, 10, 15, 20]
for n_changed in n_list:
    cadex = Cadex(model, n_changed, transform=german_data.scale, inverse_transform=german_data.unscale)
    cadexs.append(cadex)


Training s
Training loss (for one batch): 0.1951 
Training accuracy 0.919525

Training g
Training loss (for one batch): 0.9731 
Training accuracy 0.07875
Training loss (for one batch): 0.9462 
Training accuracy 0.07875

Training s
Training loss (for one batch): 0.1974 
Training accuracy 0.9197375

Training g
Training loss (for one batch): 2.2377 
Training accuracy 0.07875
Training loss (for one batch): 2.2007 
Training accuracy 0.07875

Training s
Training loss (for one batch): 0.1977 
Training accuracy 0.9195875

Training g
Training loss (for one batch): 1.5174 
Training accuracy 0.07875
Training loss (for one batch): 1.4765 
Training accuracy 0.07875

Training s
Training loss (for one batch): 0.1951 
Training accuracy 0.919825

Training g
Training loss (for one batch): 2.4546 
Training accuracy 0.07875
Training loss (for one batch): 2.4246 
Training accuracy 0.07875

Training s
Training loss (for one batch): 0.1995 
Training accuracy 0.9197

Training g
Training loss (for one batch):

Now, let's use ECE and generate up to 4 best counterfactuals:

In [42]:
from cfec.explainers import ECE
from cfec.visualization import compare
pd.set_option("display.max_rows", None)

ece = ECE(10, columns=list(german_data.X_train.columns), bces=cadexs + fimaps, dist=2, h=5, lambda_=0.001, n_jobs=1)
cfs = ece.generate(X_test)

First, let's see all 10 counterfactuals:

In [43]:
compare(X_test, ece.get_aggregated_cfs())

,X,CF1 change,CF2 change,CF3 change,CF4 change,CF5 change,CF6 change,CF7 change,CF8 change,CF9 change,CF10 change,constraint
number of attributes changed,0.0,61.000,61.000,61.000,61.000,5.0,8.0,15.0,10.0,61.000,20.0,
duration,6.0,-7.465,-6.399,-1.921,-0.011,-,-,-,-,0.017,29.097,
credit,1169.0,-13.796,-14.172,-20.289,-9.211,-,-,9630.301,13107.152,-6.172,7776.556,
installment_percent,4.0,-53.615,-51.894,-22.633,-6.486,-,-,-,-,-4.531,-,
residence_duration,4.0,-37.628,-37.335,-13.889,-1.391,-,-,-,-,-0.516,-,
age,67.0,-71.805,-71.854,-52.091,-20.988,-,-,-,-,-13.798,-,
existing_credits,2.0,-60.345,-62.533,-37.967,-17.314,-,-,-1.59,-,-11.804,-1.284,
people_maintained,1.0,20.069,20.074,4.628,2.439,-,-,-,-,1.276,-,
account_status_0..200 DM,0.0,-2.708,-0.230,-4.437,1.222,-,-,-,-,1.108,-,
account_status_< 0 DM,1.0,33.977,36.538,33.314,17.500,-,-,-,-,12.944,0.428,


And now, those selected by ECE:

In [44]:
compare(X_test, cfs)

,X,CF1 change,CF2 change,constraint
number of attributes changed,0.0,15.0,10.0,
duration,6.0,-,-,
credit,1169.0,9630.301,13107.152,
installment_percent,4.0,-,-,
residence_duration,4.0,-,-,
age,67.0,-,-,
existing_credits,2.0,-1.59,-,
people_maintained,1.0,-,-,
account_status_0..200 DM,0.0,-,-,
account_status_< 0 DM,1.0,-,-,


We can do the same with constraints:

In [46]:
fimaps = []
fimap_hyperparameters = [
    (0.1, 0.001, 0.01),
    (0.1, 0.05, 0.5),
    (0.2, 0.01, 0.1),
    (0.2, 0.08, 0.8),
    (0.5, 0.001, 0.01)
]
for tau, l1, l2 in fimap_hyperparameters:
    fimap = Fimap(tau, l1, l2, constraints=constraints, use_mapper=True)
    fimap.fit(german_data.X_train, model_predictions)
    fimaps.append(fimap)
    
cadexs = []
n_list = [10, 14, 18, 20, 25]
for n_changed in n_list:
    cadex = Cadex(model, n_changed, transform=german_data.scale, inverse_transform=german_data.unscale, constraints=constraints)
    cadexs.append(cadex)

Continous columns: ['duration', 'credit', 'installment_percent', 'residence_duration', 'age', 'existing_credits', 'people_maintained', 'account_status_0..200 DM', 'account_status_< 0 DM', 'account_status_>= 200 DM', 'account_status_no checking account', 'credit_history_all credits at this bank paid back duly', 'credit_history_critical account', 'credit_history_delay in paying off in the past', 'credit_history_existing credits paid back duly till now', 'credit_history_no credits taken', 'purpose_business', 'purpose_car (new)', 'purpose_car (used)', 'purpose_domestic appliances', 'purpose_education', 'purpose_furniture/equipment', 'purpose_others', 'purpose_radio/television', 'purpose_repairs', 'purpose_retraining', 'savings_100..500 DM', 'savings_500..1000 DM', 'savings_< 100 DM', 'savings_>= 1000 DM', 'savings_unknown/ no savings account', 'sex_status_female: divorced/separated/married', 'sex_status_male: divorced/separated', 'sex_status_male: married/widowed', 'sex_status_male: single

In [47]:
ece = ECE(4, columns=list(german_data.X_train.columns), bces=cadexs + fimaps, dist=2, h=5, lambda_=0.001, n_jobs=1)
cfs = ece.generate(X_test)

In [48]:
compare(X_test, ece.get_aggregated_cfs(), constraints=constraints)

,X,CF1 change,CF2 change,CF3 change,CF4 change,CF5 change,CF6 change,CF7 change,CF8 change,CF9 change,CF10 change,constraint
number of attributes changed,0.0,10.0,37.0,37.0,16.0,16.0,13.0,15.0,37.0,37.0,37.0,
duration,6.0,-,1.807,3.716,34.252,34.563,34.728,35.081,38.762,115.735,120.868,
credit,1169.0,9305.086,-5.741,-7.601,9154.314,9237.602,9281.553,9376.01,-2.3,48.457,46.757,
installment_percent,4.0,-,22.442,35.027,1.511,1.525,-,1.548,105.808,150.227,159.691,
residence_duration,4.0,-,18.854,31.135,1.511,1.525,-,1.548,92.268,169.492,171.072,
age,67.0,-,-,-,-,-,-,-,-,-,-,Monotonicity increasing
existing_credits,2.0,-1.536,-5.493,-8.498,-1.511,-1.525,-1.532,-1.548,-39.127,-54.312,-55.239,
people_maintained,1.0,-,13.841,22.899,0.504,0.508,-,-,60.688,85.674,90.019,
account_status_0..200 DM,0.0,-,0.275,0.275,-,-,-,-,0.275,0.275,0.275,OneHot
account_status_< 0 DM,1.0,-,-0.721,-0.721,-,-,-,-,-0.273,-0.721,-0.721,OneHot


In [49]:
compare(X_test, cfs, constraints=constraints)

,X,CF1 change,CF2 change,constraint
number of attributes changed,0.0,13.0,15.0,
duration,6.0,34.728,35.081,
credit,1169.0,9281.553,9376.01,
installment_percent,4.0,-,1.548,
residence_duration,4.0,-,1.548,
age,67.0,-,-,Monotonicity increasing
existing_credits,2.0,-1.532,-1.548,
people_maintained,1.0,-,-,
account_status_0..200 DM,0.0,-,-,OneHot
account_status_< 0 DM,1.0,-,-,OneHot


For more information on the library, see our documentation: https://counterfactuals.readthedocs.io/en/latest/